In [8]:
import os

os.getcwd()

'C:\\Users\\Eeeta\\GeeseHacks'

In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="eGjHxmLCAIMYJ89pbtFB")
project = rf.workspace("eeetan").project("doors-oyetk-kfpop-gv5s4")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to doors-1 in yolov11:: 100%|██████████| 1656/1656 [00:00<00:00, 4147.87it/s]


In [3]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')


In [5]:
results = model.train(data="C://Users/Eeeta/Geesehacks/hyacinthe/model_training/doors-1/data.yaml", epochs=20, imgsz=640,batch=8)

Ultralytics 8.3.67  Python-3.12.5 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C://Users/Eeeta/Geesehacks/hyacinthe/model_training/doors-1/data.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False

train: Scanning C:\Users\Eeeta\GeeseHacks\hyacinthe\model_training\doors-1\train\labels.cache... 720 images, 0 backgrounds, 0 corrupt: 100%|██████████| 720/720 [00:00<?, ?it/s]
val: Scanning C:\Users\Eeeta\GeeseHacks\hyacinthe\model_training\doors-1\valid\labels.cache... 67 images, 0 backgrounds, 0 corrupt: 100%|██████████| 67/67 [00:00<?, ?it/s]


Plotting labels to runs\detect\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train10
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      2.35G     0.6017     0.5804      1.149         16        640: 100%|██████████| 90/90 [00:09<00:00,  9.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00, 14.91it/s]

                   all         67         67      0.902      0.866      0.948      0.786


...

      20/20      2.34G     0.3133     0.2439     0.9831          8        640: 100%|██████████| 90/90 [00:06<00:00, 14.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00, 17.28it/s]

                   all         67         67      0.978      0.955       0.99      0.942



20 epochs completed in 0.043 hours.
Optimizer stripped from runs\detect\train10\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train10\weights\best.pt, 5.5MB

Validating runs\detect\train10\weights\best.pt...
Ultralytics 8.3.67  Python-3.12.5 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00, 12.06it/s]


                   all         67         67      0.977      0.955       0.99      0.942
Speed: 0.3ms preprocess, 1.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\train10


In [17]:
trained = YOLO("best.pt")

In [18]:
results = trained.track(source=0, stream=True, tracker='bytetrack.yaml', show=True)

In [20]:
import cv2

# Iterate over the results from the generator
for result in results:
    # Get the frame with bounding boxes and other annotations
    annotated_frame = result.plot()

    # Display the annotated frame in a window
    cv2.imshow('YOLO ByteTrack Webcam', annotated_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cv2.destroyAllWindows()

0: 480x640 (no detections), 246.2ms
0: 480x640 (no detections), 4.2ms
0: 480x640 (no detections), 6.1ms
0: 480x640 (no detections), 21.8ms
0: 480x640 (no detections), 8.0ms
0: 480x640 (no detections), 4.6ms
0: 480x640 (no detections), 5.1ms
0: 480x640 (no detections), 15.5ms
0: 480x640 (no detections), 5.2ms
0: 480x640 (no detections), 8.5ms
0: 480x640 (no detections), 7.3ms
0: 480x640 (no detections), 4.1ms
0: 480x640 (no detections), 5.2ms
0: 480x640 (no detections), 5.7ms
0: 480x640 (no detections), 14.0ms
0: 480x640 (no detections), 6.8ms
0: 480x640 (no detections), 8.3ms
0: 480x640 (no detections), 5.1ms
0: 480x640 (no detections), 9.2ms
0: 480x640 (no detections), 7.9ms
0: 480x640 (no detections), 5.6ms
0: 480x640 (no detections), 12.0ms
0: 480x640 (no detections), 8.7ms
0: 480x640 (no detections), 4.5ms
0: 480x640 (no detections), 10.2ms
0: 480x640 (no detections), 5.2ms
0: 480x640 (no detections), 6.8ms
0: 480x640 (no detections), 8.1ms
0: 480x640 (no detections), 4.2ms
0: 480x

KeyboardInterrupt: 

In [21]:
import cv2

# Open the default camera
cam = cv2.VideoCapture(0)

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

while True:
    ret, frame = cam.read()

    # Write the frame to the output file
    out.write(frame)

    # Display the captured frame
    cv2.imshow('Camera', frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break

# Release the capture and writer objects
cam.release()
out.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 